# AI. Actividad Integradora
**Alumno:** Javier E. Agostini Castilla - A00827216

**Instrucciones**

Realizar una simulación utilizando Python y el framework Mesa para visualizar un modelo de robots en un almacén capaces de mover cajas y acumularlas en pilas.

**Objetivo**



*   Observar el funcionamiento de un sistema multiagentes
*   Practicar para poder solucionar de mejor manera el reto del bloque





## Instalación de Librerias

In [310]:
!pip3 install mesa

In [311]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid
from mesa.space import SingleGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

from mesa.space import ContinuousSpace


# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import math
import random

## Declaración de Variables

In [312]:
# Dimensiones de la habitacion
M = 10
N = 10

# Cantidad de agentes (limpiadores)
agent_qty = 5

box_qty = 10

# Tiempo maximo de ejecucion en segundos
max_exe_time = 1

if M % 2 == 0:
  centers = [[M / 2 - 1, M / 2 - 1], [M / 2 - 1, M / 2], [M / 2, M / 2], [M / 2, M / 2 - 1]]
else:
  centers = [[math.floor(M / 2), math.floor(M / 2) - 1], [math.floor(M / 2), math.floor(M / 2)], [math.floor(M / 2), math.floor(M / 2) + 1]]

# centers = [[4,4], [4,5], [5,5], [5,4]]
stack_qty = 0

## Desarrollo de Clases de Modelo y Agentes

In [313]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Cleaner):
                grid[x][y] = 6
            else:
                grid[x][y] = content.live
    return grid

class Floor(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.visited = False
    self.live = 0

class Cleaner(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.box = False;

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=False,
      include_center=False)
    
    nearby_robots = self.model.grid.get_neighborhood(
      self.pos,
      moore=True,
      include_center=False)
  
    visited_neighbors = 0
    for i in possible_steps:
      this_cell = self.model.grid.get_cell_list_contents([i])
      if not this_cell[0].visited and not this_cell[0].live == 6:
        this_cell[0].visited = True
        self.model.grid.move_agent(self, i)
      else:
        visited_neighbors += 1
    
    if visited_neighbors == len(possible_steps):
      new_position = self.random.choice(possible_steps)
      this_cell = self.model.grid.get_cell_list_contents([new_position])
      while (this_cell[0].live == 6):
        new_position = self.random.choice(possible_steps)
        this_cell = self.model.grid.get_cell_list_contents([new_position])
      this_cell[0].visited = True
      self.model.grid.move_agent(self, new_position)
  
  def moveX(self):
    x, y = self.pos
    if x < centers[stack_qty][0]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0] + 1, self.pos[1])])
      if next_pos[0].live != 6:
        self.model.grid.move_agent(self, (x + 1, y))
        x += 1
    elif x > centers[stack_qty][0]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0] - 1, self.pos[1])])
      if next_pos[0].live != 6:
        self.model.grid.move_agent(self, (x - 1, y))
        x -= 1
  
  def moveY(self):
    x, y = self.pos
    if y < centers[stack_qty][1]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0], self.pos[1] + 1)])
      if next_pos[0].live != 6:
        self.model.grid.move_agent(self, (x, y + 1))
        y += 1
    elif y > centers[stack_qty][1]:
      next_pos = self.model.grid.get_cell_list_contents([(self.pos[0], self.pos[1] - 1)])
      if next_pos[0].live != 6:
        self.model.grid.move_agent(self, (x, y - 1))
        y -= 1

  def dirty(self):
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    if this_cell[0].live == 1 and self.pos != (centers[stack_qty][0], centers[stack_qty][1]) and not self.box:
      self.box = True;
      this_cell[0].live = 0
      return True
    else:
      return False
  
  def dropBox(self):
    global stack_qty
    this_cell = self.model.grid.get_cell_list_contents([self.pos])
    if this_cell[0].live < 5:
      this_cell[0].live += 1

    if this_cell[0].live == 5:
      stack_qty += 1

    self.box = False

  def step(self):
    x, y = self.pos
    if not self.dirty() and not self.box:
      self.move()
    elif self.pos == (centers[stack_qty][0], centers[stack_qty][1]) and self.box:
      self.dropBox()
    elif self.pos == (centers[stack_qty][0], centers[stack_qty][1]) and not self.box:
      self.move()
    elif x != centers[stack_qty][0]:
      self.moveX()
    elif y != centers[stack_qty][1]:
      self.moveY()
  
class Room(Model):
  def __init__(self, m, n, agent_qty, box_qty, stack_qty, max_exe_time):
    self.agent_qty = agent_qty
    # self.dirty_cell_percentage = dirty_cell_percentage
    # self.clean_cell_percentage = 1 - dirty_cell_percentage
    self.start_time = time.time()
    self.max_exe_time = max_exe_time
    self.exe_time = None
    self.grid = MultiGrid(m, n, False)
    self.schedule = SimultaneousActivation(self)

    for (content, x, y) in self.grid.coord_iter():
      floor = Floor((x, y), self)
      self.grid.place_agent(floor, (x, y))
      self.schedule.add(floor)
      
    for i in range(box_qty):
      x = np.random.choice(m)
      y = np.random.choice(n)
      this_cell = self.grid.get_cell_list_contents((x, y))
      if this_cell[0].live == 0:
        this_cell[0].live = 1

    for i in range(agent_qty):
      cleaner = Cleaner(i,self)
      x = np.random.choice(m)
      y = np.random.choice(n)
      self.grid.place_agent(cleaner, (x, y))
      self.schedule.add(cleaner)

    self.datacollector = DataCollector(
      model_reporters={"Grid": get_grid},
      agent_reporters={"Move": lambda a: getattr(a, "advance", None)}
    )
  
  def all_boxes_piled(self):
    boxes = 0
    for cell in self.grid.coord_iter():
      cell_content, x, y = cell
      for content in cell_content:
            if isinstance(content, Cleaner):
              if content.box:
                boxes = boxes + 1
      if M % 2:
        if cell_content[0].live > 0 and cell_content[0].pos != (centers[0][0], centers[0][1]) and cell_content[0].pos != (centers[1][0], centers[1][1]) and cell_content[0].pos != (centers[2][0], centers[2][1]) and cell_content[0].pos != (centers[3][0], centers[3][1]):
          boxes = boxes + 1
      else:
        if cell_content[0].live > 0 and cell_content[0].pos != (centers[0][0], centers[0][1]) and cell_content[0].pos != (centers[1][0], centers[1][1]) and cell_content[0].pos != (centers[2][0], centers[2][1]):
          boxes = boxes + 1
      
    if boxes > 0:
      return False
    else: 
      return True
    
  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()

## Ejecución

In [314]:
start_time = time.time()
model = Room(M, N, agent_qty, box_qty, stack_qty, max_exe_time)

moves = 0
while ((time.time() - start_time) < max_exe_time and not model.all_boxes_piled()):
  model.step()
  moves += 1

exe_time = str(datetime.timedelta(seconds = (time.time() - start_time)))

## Información de Ejecución

In [315]:
print('Tiempo de ejecución:', exe_time)
print('Movimientos realizados por los robots: ', moves)
print('Pilas completadas: ', stack_qty)

Tiempo de ejecución: 0:00:00.275787
Movimientos realizados por los robots:  189
Pilas completadas:  2


## Visualización

In [316]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [317]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Reds')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [318]:
anim

## Estrategia para Disminución de Tiempo


Como se puede observar en la simulación, los agentes robots, después de encontrar una caja, la mueven al centro del grid y las van apilando. Esto puede llegar a costar un mayor número de steps, debido a las distancias necesarias para ir apilando cada una de las cajas.

Es por esto que se considera que, para disminuir el tiempo necesario para apilar las cajas, así como los movimientos realizados, se podría implementar una función que, al encontrar su primer caja, el robot comience a buscar a su alrededor y vaya apilando en la posición donde encontró la primer caja. 

Aunque esta estrategia reduciría los movimientos y el tiempo de ejecución, no sería capaz de ordenar las pilas de manera adyacente.

## Conclusión

En conclusión, esta actividad me permitió entender de mejor manera la implementación de un sistema multiagentes, así como sus diferentes usos y alcances. 

La actividad tuvo un grado de dificultad alto pero creo que se pudo desarrollar una buena solución. Espero poder seguir aprendiendo sobre el tema y poder seguir desarrollando mis habilidades en herramientas como esta y Unity.